<a href="https://colab.research.google.com/github/ShivamThapa243/Information-Retrieval/blob/main/unigram_inverted_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UNIGRAM INVERTED INDEX

In [1]:
# importing files from drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Building Unigram Inverted Index

---


Structure of inverted index:
*   word1: {counts: x, documents: [doc1, dox2, doc3...]}
*   word2: {counts: y, documents: [doc3, doc4, doc5...]}
*   ...


    

In [35]:
import os

def unigram_inverted_index_builder(dataset_directory):
  unigram_inverted_index = {}
  list_file = os.listdir(dataset_directory)

  # Iterating through each file present in the directory
  for filename in list_file:
    if filename.endswith(".txt"):
      # Reading the content of the file
      file_path = os.path.join(dataset_directory, filename)
      with open(file_path, 'r') as file:
        content = file.read()

      # tokenizing the documnet to get unique tokens
      content_list = content.split()
      unique_content = set(content_list)

      # Updating the unigram inverted index
      for token in unique_content:
        # if the token is already in the inverted_index
        if token in unigram_inverted_index:
          unigram_inverted_index[token]['count'] += 1
          if filename not in unigram_inverted_index[token]['documents']:
            unigram_inverted_index[token]['documents'].append(filename)
        else:
          unigram_inverted_index[token] = {'count' : 1, 'documents': [filename]}
  return unigram_inverted_index


Invoking the unigram_inverted_index_builder function to build the inverted index

In [36]:
dataset_directory = "/content/drive/MyDrive/Information Retrieval/preprocessed_data"

unigram_inverted_index = unigram_inverted_index_builder(dataset_directory)

text_file_name = "unigram_inverted_index_dataset.txt"
text_file_directory = "/content/drive/MyDrive/Information Retrieval"
text_file = os.path.join(text_file_directory, text_file_name)

with open(text_file, 'w') as file:
  for term, info in unigram_inverted_index.items():
    file.write(f"{term}: {info}\n")

print("Unigram inverted index created")

Unigram inverted index created
